In [ ]:
import os,sys
sys.path.append(os.path.abspath(".."))

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from core.data import *
from scripts.process_data import make_final_df
from core.utils import seed_all
import seaborn as sns
# sns.set_theme(style="darkgrid")
seed = 1
palette = sns.palettes.color_palette("bright", as_cmap=True)
sns.set_palette(palette=palette)
seed_all(seed)

In [ ]:
from api import Recommender,Request
Recommender.get_instance().recommend(Request(
    userId=1,
    movieIds=[],
    genres=[],
    model="DLRM",
    start=0,
    count=None,
    relation="and",
))

In [ ]:
from core.suggester import load_suggester
suggester = load_suggester()

In [ ]:
movieIds = suggester.suggest(10,30,genres=["Fantasy"],relation="or",add_closest_users=True,deterministic=False)
ratings.loc[ratings.movie.isin(movieIds)].movie_genres

In [ ]:
movies, ratings = load_ds()
movies, users, movie_cols, user_cols = preprocess(movies, ratings,keep_title=True,keep_genres=True)
(movie_cat_cols, movie_num_cols), (user_cat_cols, user_num_cols) = movie_cols, user_cols

n_users = users.user.nunique()
n_movies = movies.movie.nunique()

In [ ]:
all_genres = exclude(movie_cat_cols,"movie","year_bin")
count1 = movies[all_genres].astype(int).sum().values
count2 = movies[all_genres].loc[movies.movie_avg_rating>0].astype(int).sum().values
counts = pd.DataFrame(dict(genre=all_genres,all=count1,positive_rating=count2))
counts.head(1)

In [ ]:
def pie(count,labels,title="",ax=None):
    total = sum(count)
    def my_fmt(x):
        s = f'{x:.0f}%'
        if x>5:
            s += f"\n({total*x/100:.0f})"
        return s
    if ax is None: ax = plt.subplot()
    ax.pie(count, labels=labels, autopct=my_fmt)
    ax.set_title(title)
fig,axes = plt.subplots(1,2,figsize=(15,7))
for i, ax in zip(["all", "positive_rating"], axes):
    pie(counts[i], counts.genre, ax=ax)
fig.suptitle("Genre distribution");
fig.tight_layout()

In [ ]:
means = [movies.loc[(movies[g] == 1) & (movies.movie_avg_rating>0)].movie_avg_rating.mean() for g in all_genres]
sns.barplot(pd.DataFrame(dict(avg_rating=means, labels=all_genres)),x="avg_rating", y="labels", palette=palette)
plt.tight_layout()

In [ ]:
_, (ax1,ax2) = plt.subplots(1,2,figsize=(15,5))
sns.histplot(data=movies,x="movie_avg_rating",ax=ax1)
sns.histplot(data=ratings, x="rating", kde=True, ax=ax2)
plt.tight_layout()

In [ ]:
nrated = (movies.movie_avg_rating > 0).rename("is_rated").value_counts().reset_index()
pie(nrated["count"], nrated.is_rated,title="Movie with at least 1 rating.")
nrated

In [ ]:
a = movies.year_bin.value_counts().sort_index().reset_index()
pie(a["count"],a.year_bin,"Movies production year")
plt.tight_layout()

In [ ]:
movieecols = exclude(movies.columns.tolist(), "__movieId", "title","genres","main_genre")
user_colss = exclude(users.columns.tolist(), "__userId")